## LightGCN with Sign-aware  BPR loss

## Import Packages

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from scipy import sparse
import os
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn.functional as F
from torch import Tensor
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.data import Data

### Loading data

In [2]:
df_val = pd.read_csv('df_val.csv')

In [3]:
df_train = pd.read_csv('quadruplet.csv')

In [4]:
R = load_npz('R_train.npz')

In [5]:
n_users,n_movies = R.shape

### Prepare dataset for PyTorch model

In [6]:
class MovieDataset(Dataset):
    def __init__(self,df):
        self.df  = df
        
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,idx):
        
        u = torch.tensor(self.df.iloc[idx,0],dtype=torch.long)
        p = torch.tensor(self.df.iloc[idx,1],dtype=torch.long)
        n = torch.tensor(self.df.iloc[idx,3],dtype=torch.long)
        r = torch.tensor(self.df.iloc[idx,2],dtype=torch.long)
        
        return (u,p,n,r)

In [7]:
train_dataset = MovieDataset(df_train)

In [8]:
BATCH_SIZE = 2048
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         )

### Constructing accuracy function

In [9]:
one_idxs=[]
minus_one_idxs=[]
hold_out=[]
hold_out_minus=[]

In [10]:
for i in range(n_users):
    one_idxs.append(np.where(R[[i],:].toarray()[0] == 1)[0])
    minus_one_idxs.append(np.where(R[[i],:].toarray()[0] == -1)[0])
    hold_out.append(df_val.query(f"userId=={i} & rating==1").movieId.values)
    hold_out_minus.append(df_val.query(f"userId=={i} & rating==-1").movieId.values)

In [11]:
def accuracy_func(E,k=10):
    E = E.to('cpu').detach()
    ue, ui = torch.split(E,[n_users,n_movies])
    S = torch.matmul(ue,ui.T).numpy()
    accuracy = []
    for i in range(n_users):
        output = S[i]
        np.put(output,one_idxs[i],-np.inf)
        np.put(output,minus_one_idxs[i],-np.inf)
        c = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out[i]))
        nc = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out_minus[i]))
#         acc = np.max([0,(c-nc)/(np.min([k,len(hold_out[i])+1]))]) ## Recal@K
        acc = np.max([0,(c-nc)/k]) ## HR@K
        accuracy.append(acc)
    return np.mean(accuracy)

### LightGCN model + sBPR Loss

In [12]:
class LightGCN(MessagePassing):
    def __init__(self,num_users=n_users,
                 num_items=n_movies,
                embedding_size=64,
                n_layers=3):
        super().__init__(aggr='add')
        self.n_users = num_users
        self.n_items = num_items
        self.embedding_size = embedding_size
        self.n_layers = n_layers
        self.E = nn.Parameter(torch.empty(n_users + n_movies,self.embedding_size))
        nn.init.xavier_normal_(self.E.data)
        
        
        
    def forward(self,edge_index):
        
        row, col = edge_index
        deg = degree(col, self.E.size(0), dtype=self.E.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        
        B=[]
        B.append(self.E)
        x = self.E

        for i in range(self.n_layers):
            x = self.propagate(edge_index,x=x,norm=norm)
            B.append(x)
        
        final_embedding = sum(B)/len(B)
        first_embedding = B[0]
        return final_embedding, first_embedding
    
    def message(self,x_j,norm):
        return norm.view(-1,1) * x_j
    def update(self,inputs: Tensor) -> Tensor:
        return inputs

In [13]:
class sBPR(nn.Module):
    def __init__(self,n_users,n_items,reg=0.01):
        super().__init__()
        
        self.reg = reg
        self.n_users = n_users
        self.n_items = n_items
        
    def forward(self,E,E0,u,p,n,r):
        self.E = E
        self.E0 = E0
        self.user_embedding, self.item_embedding = torch.split(self.E,[self.n_users,self.n_items])
        self.user_embedding_0, self.item_embedding_0 = torch.split(self.E0,[self.n_users,self.n_items])
        u_ = self.user_embedding[u].to(device)
        p_ = self.item_embedding[p].to(device)
        n_ = self.item_embedding[n].to(device)
        r_ = r.to(device)
        u0 = self.user_embedding_0[u].to(device)
        p0 = self.item_embedding_0[p].to(device)
        n0 = self.item_embedding_0[n].to(device)
        
        
        
        positive_interaction = torch.mul(u_, p_).sum(dim=1)
        negative_interaction = torch.mul(u_, n_).sum(dim=1)
        sign_delta = ((-1/2*torch.sign(r_)+3/2)*positive_interaction) - negative_interaction
        
        log_prob = F.logsigmoid(sign_delta).sum()
        regularization = self.reg * (u0.norm(dim=1).pow(2).sum() + p0.norm(dim=1).pow(2).sum() + n0.norm(dim=1).pow(2).sum())
        return (-log_prob + regularization)/len(u_)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
model = LightGCN().to(device)

In [16]:
loss = sBPR(n_users,n_movies).to(device)

In [17]:
optimizer = optim.Adam(model.parameters(),lr=0.005)

In [18]:
row, col = R.nonzero()

In [19]:
edge_user = torch.tensor(row,dtype=torch.long)
edge_item = torch.tensor(col,dtype=torch.long) + n_users

In [20]:
edge_ = torch.stack((torch.cat((edge_user,edge_item),0),torch.cat((edge_item,edge_user),0)),0)

In [21]:
A_edge_index = Data(edge_index=edge_)

In [22]:
epochs = 10

for epoch in range(epochs):
    model.train()
    train_losses = []
    for i,(u,p,n,r) in enumerate(train_loader):
        
        E,E_0 = model(A_edge_index.edge_index.to(device)) 
        cost = loss(E,E_0,u,p,n,r)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        train_losses.append(cost.item())
    model.eval()
    E,_ = model(A_edge_index.edge_index.to(device))
    acc = accuracy_func(E,10)
    
    print(f"Epoch {epoch + 1},train loss: {torch.tensor(train_losses).mean():.4f}, val accuracy: {acc:.4f}")


Epoch 1,train loss: 0.3831, val accuracy: 0.1638
Epoch 2,train loss: 0.3608, val accuracy: 0.1639
Epoch 3,train loss: 0.3607, val accuracy: 0.1646
Epoch 4,train loss: 0.3595, val accuracy: 0.1675
Epoch 5,train loss: 0.3567, val accuracy: 0.1731
Epoch 6,train loss: 0.3556, val accuracy: 0.1757
Epoch 7,train loss: 0.3554, val accuracy: 0.1766
Epoch 8,train loss: 0.3553, val accuracy: 0.1767
Epoch 9,train loss: 0.3553, val accuracy: 0.1769
Epoch 10,train loss: 0.3552, val accuracy: 0.1765
